In [1]:
import re

import pandas as pd
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sakya\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
comments = pd.read_csv('CommentsEpic.csv')

# Sample sentiment analysis
1. Using prebuilt packages
    - Data cleaning and exploration
    - Vader and Textblob  </br>
2. Training own classifier
    - Preprocessing
    - Naive Bayes

## __1.__ __Using prebuilt packages__

 - ### __Data cleaning and exploration__

In [3]:
sample = comments.Reply.iloc[10]
sample

'why is this same meme everywhere?'

In [4]:
re.sub(r'\(?http\S+', '', sample)

'why is this same meme everywhere?'

Getting rid of \n, \t... etc characters that does not help with prediction but might link 2 words together and make them indistinguishable from one another

In [5]:
sample = comments.Reply.iloc[9]
sample

'Lol but true'

In [6]:
word_tokenized = nltk.word_tokenize(sample)
word_tokenized

['Lol', 'but', 'true']

In [7]:
' '.join(word_tokenized)

'Lol but true'

#### Bigrams

In [8]:
sample = ['more', 'is', 'said', 'than', 'done', 'is', 'said', 'than']

In [9]:
from nltk.collocations import *

bigram_measures = nltk.collocations.BigramAssocMeasures()

finder = BigramCollocationFinder.from_words(sample)

# only bigrams that appear 2+ times
finder.apply_freq_filter(2) 

# return the 10 n-grams with the highest PMI
print (finder.nbest(bigram_measures.pmi, 10) )

[('is', 'said'), ('said', 'than')]


Let's try it on first 10000 rows of our text

In [10]:
all_text = comments.iloc[:10000,:].Reply.str.cat(sep='. ')

In [11]:
all_text = nltk.word_tokenize(all_text)

In [12]:
def common_bigrams(tokenized_text, min_freq, top_n):
    bigram_measures = nltk.collocations.BigramAssocMeasures()
    finder = BigramCollocationFinder.from_words(tokenized_text)
    finder.apply_freq_filter(min_freq) 
    finder.nbest(bigram_measures.pmi, top_n)
    return finder.nbest(bigram_measures.pmi, top_n)

print(common_bigrams(all_text, 10, 10))

[('OTHERS', 'CLICKED'), ('Theft', 'Auto'), ('Starter', 'Pack'), ('Rocket', 'League'), ('Grand', 'Theft'), ('Unreal', 'Engine'), ('task', 'manager'), ('Criminal', 'Enterprise'), ('shopping', 'cart'), ('CLICKED', 'THIS')]


#### Wordcloud

In [13]:
import wordcloud
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt

sample = comments.iloc[:10000,:].Reply.str.cat(sep='. ')

In [14]:
import random
def grey_color_func(word, font_size, position, orientation, random_state=None, **kwargs):
    return "hsl(0, 0%%, %d%%)" % random.randint(1, 20)

In [15]:
stp = STOPWORDS.copy()
wc = WordCloud(background_color="white", max_words=50,  stopwords=stp)
# generate word cloud
wc.generate(sample)

plt.figure(figsize=(10,20))
plt.imshow(wc.recolor(color_func=grey_color_func, random_state=3))
plt.show()

- ### __Vader and Textblob__

Vader package first

In [16]:
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer, VaderConstants

#FIRST, we initialize VADER so we can use it within our Python script
sid = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\sakya\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


We can add sentiment scores to words manually if we want

In [17]:
new_words = {

}

sid.lexicon.update(new_words)

In [18]:
sid.polarity_scores('hello there')

{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0}

In [19]:
#As this is for demonstration purposes only - instead of doing the scoring over 200,000+ rows, we will use a subset
df = comments #[comments.Key<101]
df.shape

(24647, 4)

In [20]:
#The cleaning steps as outlined earlier
df.Reply = df.Reply.apply(lambda x: re.sub(r'\(?http\S+', '', x))
df.Reply = df.Reply.apply(lambda x: ' '.join(nltk.word_tokenize(x)))

In [21]:
df[['neg','neu','pos','compound']] = df.Reply.apply(lambda x: pd.Series(sid.polarity_scores(x)))

In [22]:
df.head(3)

Reply  Upvote  \
0  I just did my first purchase from epic , I got...    25.0   
1  Actually i already bought 3 games on epic : Th...     8.0   
2              me who got the 80th game for free : )     7.0   

                  Time  Key    neg    neu    pos  compound  
0  2020-05-22 12:37:51  1.0  0.140  0.860  0.000   -0.4168  
1  2020-05-22 12:45:06  1.0  0.185  0.736  0.079   -0.6369  
2  2020-05-22 14:11:36  1.0  0.000  0.680  0.320    0.5106

Textblob package now

In [23]:
from textblob.sentiments import NaiveBayesAnalyzer, PatternAnalyzer
from textblob import Blobber
tba = Blobber(analyzer=NaiveBayesAnalyzer())

In [24]:
nltk.download('movie_reviews')
tba('My life is a frictionless slide downhill').sentiment

[nltk_data] Downloading package movie_reviews to
[nltk_data]     C:\Users\sakya\AppData\Roaming\nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!


Sentiment(classification='neg', p_pos=0.374939442175929, p_neg=0.6250605578240714)

In [25]:
#As this is for demonstration purposes only - we will only use a subset
df = comments #[comments.Key<101]

In [26]:
#The cleaning steps as outlined earlier
df.Reply = df.Reply.apply(lambda x: re.sub(r'\(?http\S+', '', x))
df.Reply = df.Reply.apply(lambda x: ' '.join(nltk.word_tokenize(x)))

In [27]:
df[['Classification','positive','negative']] = df.Reply.apply(lambda x: pd.Series({'Classification':tba(x).sentiment[0],
                                                                        'positive':tba(x).sentiment[1],
                                                                        'negative':tba(x).sentiment[2]}))

In [28]:
df.head(3)

Reply  Upvote  \
0  I just did my first purchase from epic , I got...    25.0   
1  Actually i already bought 3 games on epic : Th...     8.0   
2              me who got the 80th game for free : )     7.0   

                  Time  Key    neg    neu    pos  compound Classification  \
0  2020-05-22 12:37:51  1.0  0.140  0.860  0.000   -0.4168            pos   
1  2020-05-22 12:45:06  1.0  0.185  0.736  0.079   -0.6369            pos   
2  2020-05-22 14:11:36  1.0  0.000  0.680  0.320    0.5106            neg   

   positive  negative  
0  0.742065  0.257935  
1  0.866839  0.133161  
2  0.497501  0.502499

In [29]:
df.describe()

Upvote           Key           neg           neu           pos  \
count  24647.000000  24647.000000  24647.000000  24647.000000  24647.000000   
mean       2.934353    193.677324      0.066099      0.784138      0.145948   
std        6.912765    140.363728      0.122869      0.219479      0.194993   
min      -46.000000      1.000000      0.000000      0.000000      0.000000   
25%        1.000000     73.000000      0.000000      0.684000      0.000000   
50%        1.000000    161.000000      0.000000      0.823000      0.086000   
75%        3.000000    310.000000      0.095000      1.000000      0.216000   
max      217.000000    499.000000      1.000000      1.000000      1.000000   

           compound      positive      negative  
count  24647.000000  2.464700e+04  2.464700e+04  
mean       0.165784  5.643509e-01  4.356491e-01  
std        0.443218  2.655260e-01  2.655260e-01  
min       -0.993200  1.682455e-07  2.362975e-21  
25%        0.000000  3.864079e-01  2.264119e-01  
50%        0.000000  5.501663e-01  4.498337e-01  
75%        0.510600  7.735881e-01  6.135921e-01  
max        0.999100  1.000000e+00  9.999998e-01

In [30]:
df['Classification'].value_counts()

pos    15110
neg     9537
Name: Classification, dtype: int64

In [31]:
##Import Trained Classifier Module from my other notebook

In [32]:
import import_ipynb
%run Kaggle_Trained_Classifier.ipynb

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sakya\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [33]:
Z = vectorizer.transform(df['Reply'])

In [34]:
predicted2 = nb.predict(Z)

In [35]:
df2 = df

In [36]:
df2['classification from kaggel'] = predicted2

In [37]:
df2.head()

Reply  Upvote  \
0  I just did my first purchase from epic , I got...    25.0   
1  Actually i already bought 3 games on epic : Th...     8.0   
2              me who got the 80th game for free : )     7.0   
3     I 'm watching the office as I upvote this post    17.0   
4  damn , so true 😂 They earn a lot by showing to...     3.0   

                  Time  Key    neg    neu    pos  compound Classification  \
0  2020-05-22 12:37:51  1.0  0.140  0.860  0.000   -0.4168            pos   
1  2020-05-22 12:45:06  1.0  0.185  0.736  0.079   -0.6369            pos   
2  2020-05-22 14:11:36  1.0  0.000  0.680  0.320    0.5106            neg   
3  2020-05-22 10:17:32  1.0  0.000  1.000  0.000    0.0000            neg   
4  2020-05-22 17:11:35  1.0  0.112  0.623  0.265    0.5709            neg   

   positive  negative classification from kaggel  
0  0.742065  0.257935                    neutral  
1  0.866839  0.133161                    neutral  
2  0.497501  0.502499                    neutral  
3  0.381604  0.618396                    neutral  
4  0.210738  0.789262                   positive

In [38]:
predicted2.shape

(24647,)

In [39]:
df.shape

(24647, 12)

In [40]:
df['classification from kaggel'].value_counts()

neutral     13619
negative     5812
positive     5216
Name: classification from kaggel, dtype: int64

In [41]:
df2.head(10)

Reply  Upvote  \
0  I just did my first purchase from epic , I got...    25.0   
1  Actually i already bought 3 games on epic : Th...     8.0   
2              me who got the 80th game for free : )     7.0   
3     I 'm watching the office as I upvote this post    17.0   
4  damn , so true 😂 They earn a lot by showing to...     3.0   
5                              HAHAHAHA Receipt GANG     2.0   
6  I recently found out about Epic Games so I mis...     2.0   
7                      Damn . Actually that 's me.😁😁     1.0   
8  i got ac origins ( $ 60 ) for 5 dollars today ...     1.0   
9                                       Lol but true     1.0   

                  Time  Key    neg    neu    pos  compound Classification  \
0  2020-05-22 12:37:51  1.0  0.140  0.860  0.000   -0.4168            pos   
1  2020-05-22 12:45:06  1.0  0.185  0.736  0.079   -0.6369            pos   
2  2020-05-22 14:11:36  1.0  0.000  0.680  0.320    0.5106            neg   
3  2020-05-22 10:17:32  1.0  0.000  1.000  0.000    0.0000            neg   
4  2020-05-22 17:11:35  1.0  0.112  0.623  0.265    0.5709            neg   
5  2020-05-22 19:17:19  1.0  0.000  1.000  0.000    0.0000            pos   
6  2020-05-29 06:43:21  1.0  0.225  0.579  0.196   -0.0171            neg   
7  2020-05-22 17:04:02  1.0  0.403  0.597  0.000   -0.4019            neg   
8  2020-05-22 18:51:45  1.0  0.000  0.642  0.358    0.5994            pos   
9  2020-06-15 16:42:31  1.0  0.000  0.152  0.848    0.6808            neg   

   positive  negative classification from kaggel  
0  0.742065  0.257935                    neutral  
1  0.866839  0.133161                    neutral  
2  0.497501  0.502499                    neutral  
3  0.381604  0.618396                    neutral  
4  0.210738  0.789262                   positive  
5  0.723022  0.276978                   positive  
6  0.081125  0.918875                   positive  
7  0.424121  0.575879                   negative  
8  0.706512  0.293488                   positive  
9  0.212236  0.787764                   positive

Using the Textblob package, we can see that the Epic Games Store reddit has a mixed sentiment response on posts. 39% bad sentimes and 61% good sentiment.

Using our own trained classification module, there seems to be more Negative sentiments compared to Positive ones. 23% Negative response and 21% Positive sentimes. But if we set aside the Neutral Sentimes, this looks like a very mixed sentiment.

In [43]:
df['classification from kaggel'].value_counts(normalize=True) * 100

neutral     55.256218
negative    23.580963
positive    21.162819
Name: classification from kaggel, dtype: float64

In [44]:
df['Classification'].value_counts(normalize=True) * 100

pos    61.305636
neg    38.694364
Name: Classification, dtype: float64